In [2]:
snakemake.input.keys()

In [3]:
from fenics import *
from multiphenics import *
import numpy as np
from braininversion.IOHandling import (read_mesh_from_h5, write_to_xdmf, 
                                       xdmf_to_unstructuredGrid, read_xdmf_timeseries)
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                                           plot_pressures_and_forces_cross_section,
                                           extract_cross_section, style_dict)
import matplotlib.pyplot as plt
import yaml
import pyvista as pv
from pathlib import Path


In [4]:
mesh_file = snakemake.input["subdomain_file"]
sim_file = snakemake.input["sim_results"]
sim_config_file = snakemake.input["sim_config_file"]

mesh_name = f"{Path(mesh_file).parent}/{Path(mesh_file).stem}"
mesh_config_file = f"{mesh_name}_config.yml"

boundary_file = f"{mesh_name}_boundaries.xdmf"
label_boundary_file = f"{mesh_name}_label_boundaries.xdmf"
label_file = f"{mesh_name}_labels.xdmf"

with open(sim_config_file) as conf_file:
    sim_config = yaml.load(conf_file, Loader=yaml.FullLoader)
    
with open(mesh_config_file) as conf_file:
    mesh_config = yaml.load(conf_file, Loader=yaml.FullLoader)

T = sim_config["T"]
num_steps = sim_config["num_steps"]
# subdomain ids
fluid_id = 2
porous_id = 1

# boundary ids
interface_id = 1
rigid_skull_id = 2
spinal_outlet_id = 3
fixed_stem_id = 4


mmHg2Pa = 132.32
v_order = 2

In [5]:
ventricle_probe = [Point(mesh_config["ventricle_probe"])]
sas_probe = [Point(mesh_config["sas_probe"])]
parenchyma_probe = [Point(mesh_config["parenchyma_probe"])]


dt = T/num_steps
times = np.linspace(0, T, num_steps + 1)
infile_mesh = XDMFFile(mesh_file)
mesh = Mesh()
infile_mesh.read(mesh)
gdim = mesh.geometric_dimension()
subdomain_marker = MeshFunction("size_t", mesh, gdim)
infile_mesh.read(subdomain_marker)#, "subdomains")

label_marker = MeshFunction("size_t", mesh, gdim)
infile_label = XDMFFile(label_file)
infile_label.read(label_marker)#, "subdomains")


boundary_marker = MeshFunction("size_t", mesh, gdim - 1, 0)
boundary_infile = XDMFFile(boundary_file)
boundary_infile.read(boundary_marker)
boundary_infile.close()

label_boundary_marker = MeshFunction("size_t", mesh, gdim - 1, 0)
label_boundary_infile = XDMFFile(label_boundary_file)
label_boundary_infile.read(label_boundary_marker)
label_boundary_infile.close()

fluid_submesh = SubMesh(mesh, subdomain_marker, fluid_id)
por_submesh = SubMesh(mesh, subdomain_marker, porous_id)

In [6]:
class subdomainFilter(UserExpression):
    def __init__(self, subdomain_marker, subdomain_id, **kwargs):
        self.marker = subdomain_marker
        self.domain_id = subdomain_id
        super().__init__(**kwargs)


    def eval_cell(self, values, x, cell):
            if self.marker[cell.index] == self.domain_id:
                values[0] = 1
            else:
                values[0] = np.nan

fluid_filter = subdomainFilter(subdomain_marker, fluid_id, degree=0)
por_filter = subdomainFilter(subdomain_marker, porous_id, degree=0)

DG = FunctionSpace(mesh, "DG", 0)
fluid_filter = interpolate(fluid_filter, DG)
fluid_filter.set_allow_extrapolation(True)
por_filter = interpolate(por_filter, DG)
por_filter.set_allow_extrapolation(True)

In [7]:
V = VectorFunctionSpace(mesh, "CG", v_order)

W = FunctionSpace(mesh, "CG", 1)

names = {"pF":W, "pP":W, "phi":W,"d":V,"u":V}

#names = {"pF":W_fluid, "pP":W_por, "phi":W_por,"d":V_por,"u":V_fluid}
domains = {"pF":"fluid", "pP":"porous", "phi":"porous",
         "d":"porous", "u":"fluid"}

infile = XDMFFile(sim_file)

In [8]:
results = {n:[] for n in names}
for n, space in names.items():
    for i in range(num_steps + 1):
        f = Function(space)
        if domains[n] == "fluid":
            infile.read_checkpoint(f, n, i)
        elif domains[n] == "porous":
            infile.read_checkpoint(f, n, i)
        else:
            print("error!")
        results[n].append(f)

infile.close()


In [9]:
# plot pressure evolution at different probe points
pP_series = results["pP"]
pF_series = results["pF"]
phi_series = results["phi"]

pF_sas = extract_cross_section(pF_series, sas_probe).flatten()/mmHg2Pa
pF_ventricle = extract_cross_section(pF_series, ventricle_probe).flatten()/mmHg2Pa
pP_parenchyma = extract_cross_section(pP_series, parenchyma_probe).flatten()/mmHg2Pa
phi_parenchyma = extract_cross_section(phi_series, parenchyma_probe).flatten()/mmHg2Pa

plt.figure(figsize=(10,8))
plt.plot(times, pF_ventricle, label="ventricle")
plt.plot(times, pP_parenchyma, label="parenchyma fluid")
plt.plot(times, phi_parenchyma, label="parenchyma tot")
plt.plot(times, pF_sas, label="SAS")

plt.legend()
plt.grid()
plt.xlabel("t [s]")
plt.ylabel("p in mmHg")

In [10]:
# compute pressure gradient
dist = np.array(mesh_config["ventricle_probe"]) - np.array(mesh_config["sas_probe"])
dist = np.linalg.norm(dist)
diff = pF_ventricle - pF_sas
grad = diff/dist
plt.figure(figsize=(10,8))
plt.plot(times, grad , label="ventricle")

plt.legend()
plt.grid()
plt.xlabel("t [s]")
plt.ylabel("pressure grad in mmHg/m")

In [11]:
# plot cross section through the domain
x_coords = np.linspace(0.0, 0.12, 1000)
if gdim==2:
    cross_points = [Point(x, 0) for x in x_coords]
elif gdim==3:
    cross_points = [Point(x, 0, 0) for x in x_coords]

pF_cross = extract_cross_section(pF_series, cross_points, filter_function=fluid_filter)/mmHg2Pa
pP_cross = extract_cross_section(pP_series, cross_points, filter_function=por_filter)/mmHg2Pa
phi_cross = extract_cross_section(phi_series, cross_points, filter_function=por_filter)/mmHg2Pa


for i in np.arange(1, num_steps, 5):
    plt.figure(figsize=(10,8))
    plt.plot(x_coords, pF_cross[i,:], ".-",label="fluid pressure")
    plt.plot(x_coords, pP_cross[i,:], ".-", label="porous fluid pressure")
    plt.plot(x_coords, phi_cross[i,:], ".-", label="total pressure")

    plt.legend()
    plt.grid()
    plt.title(f"t = {(i)*dt:.3f}")
    plt.xlabel("x in m")
    plt.ylabel("p in mmHg")

In [12]:
# compute outflow into spinal coord 
ds_outflow = Measure("ds", domain=mesh, subdomain_data=boundary_marker, subdomain_id=spinal_outlet_id)
n = FacetNormal(mesh)

m3tomL = 1e6
outflow = np.array([assemble(dot(u,n)*ds_outflow) for u in results["u"]])
plt.figure(figsize=(10,8))
plt.plot(times, outflow*m3tomL, label="outflow into spinal coord")
plt.legend()
plt.grid()
plt.xlabel("time in s")
plt.ylabel("flowrate in mL/ s")

In [13]:
cum_outflow = np.cumsum(outflow)*dt
plt.figure(figsize=(10,8))
plt.plot(times, cum_outflow*m3tomL, label="cumulative outflow into spinal coord")
plt.legend()
plt.grid()
plt.xlabel("time in s")
plt.ylabel("V in mL")

In [14]:
# compute transitional flow in and out of parenchyma
[u.set_allow_extrapolation(True) for u in results["u"]]
ds_interf = Measure("dS", domain=mesh, subdomain_data=boundary_marker, subdomain_id=interface_id)
dx = Measure("dx", domain=mesh, subdomain_data=subdomain_marker)
transitional_flow = np.array([assemble(dot(u("-"), n("-"))*ds_interf + Constant(0.0)*dx) for u in results["u"]])
plt.figure(figsize=(10,8))
plt.plot(times, transitional_flow*m3tomL, label="outflow into spinal coord")
plt.legend()
plt.grid()
plt.xlabel("time in s")
plt.ylabel("flowrate in mL/ s")